# Economics Tie-In

It's considered general knowledge that the worse state the economy/country is in, the better the entertainment industry does. But is this true? Can we see any sort of relationship between the state of the film industry and the standings of average means in the US?

To find out, we will be using two datasets (one for average wages by metric, and one for unemployment rates by metric) to create one economic DataFrame (DF) from which we can draw our comparrisons.

In [1]:
# Importing dependencies
import pandas as pd
import matplotlib.pyplot as plt

# Data

---

The following datasets are courtesy of __[Kaggle](https://www.kaggle.com/)__.

### __['...CCI_OECD.csv'](https://www.kaggle.com/datasets/iqbalsyahakbar/cci-oecd)__

*renamed from `DP_LIVE_16112023095843236.csv`*

Per the Organisation for Economic Co-operation and Development (OECD);

* The Consumer Confidence Indicator (CCI) is an indication of developments for future households' consumption and saving based on expected financial situation, sentiment regarding the general economic situation, employment status, and capacity for savings
* An indicator above `100` indicates an optimistic outlook and a greater likliehood to spend money over cautious saving
* An indicator below `100` indicates a pessimistic outlook and both a higher likeliehood to save money and a lower tendency to consume

### __['...US_inflation_rates.csv'](https://www.kaggle.com/datasets/pavankrishnanarne/us-inflation-dataset-1947-present)__

Per the dataset description;

* The Consumer Price Index (CPI) is a critical economic indicator for measuring the purchasing power of money over time, measuring the average change over time in the prices paid by urban consumers for goods and services
* The CPI is the value at the end of the respective month

### __['...public_debt.csv'](https://www.kaggle.com/datasets/pavankrishnanarne/us-public-debt-quarterly-data-1996-present)__

Per the dataset description;

* Public debt, or government debt, represents the total outstanding bonds and other securities of a country's central government
* The provided `value` represents the quarterly US public debt in billions of US Dollars (USD)

---

The following datasets are courtesy of the __[Economic Policy Institute’s (EPI) State of Working America Data Library](https://www.epi.org/data/)__.

### __['...Medianaverage hourly wages.csv'](https://www.epi.org/data/#?subject=wage-avg)__

Per EPI description;

* Wages adjusted to 2023 dollars by the Consumer Price Index for All Urban Consumers Research Series
* `Median Wage` is the hourly wage in the middle of the wage distribution
* `Average Wage` is the calculated mean of all wages

### __['...Unemployment.csv'](https://www.epi.org/data/#?subject=unemp)__

Per EPI description;

* Unemployment is the share of the labor force wihout a job
* Monthly percentages calculated as a rolling 12-month average (mean)

In [2]:
# Reading in data
df_hourly_wages = pd.read_csv("./Resources/EPI Data Library - Medianaverage hourly wages.csv")
df_unemp = pd.read_csv("./Resources/EPI Data Library - Unemployment.csv")
df_cci = pd.read_csv("./Resources/CCI_OECD.csv")
df_inflation = pd.read_csv("./Resources/US_inflation_rates.csv")
df_public_debt = pd.read_csv("./Resources/public_debt.csv")

# CCI

#### Preprocessing of the `CCI_OECD.csv` dataset

This dataset came with internaitonal records and uneeded features, so only records for US CCI will be retained. Once those records have been selected, the resulting DF will need to be prepared for concatenation with the remainined economic datasets. To do this, the `TIME` feature will be converted to datetime and set as the index.

In [3]:
# Viewing `df_cci`
df_cci.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-01,101.4526,NaN
1,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-02,101.4332,NaN
2,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-03,101.2753,NaN
3,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-04,101.0106,NaN
4,NLD,CCI,AMPLITUD,LTRENDIDX,M,1973-05,100.8543,NaN


#### Beginning of limited EDA

In [4]:
# Beginning EDA on `df_cci`
df_cci.describe()

,Value,Flag Codes
count,21552.000000,0.0
mean,100.017942,NaN
std,1.973255,NaN
min,86.618690,NaN
25%,98.897823,NaN
50%,100.279300,NaN
75%,101.263550,NaN
max,108.773100,NaN


In [5]:
# Continuing EDA
df_cci.shape

(21552, 8)

In [6]:
# Continuing EDA
df_cci.dtypes

LOCATION       object
INDICATOR      object
SUBJECT        object
MEASURE        object
FREQUENCY      object
TIME           object
Value         float64
Flag Codes    float64
dtype: object

In [7]:
# Continuing EDA
df_cci['LOCATION'].unique()

array(['NLD', 'CHE', 'FRA', 'POL', 'CZE', 'JPN', 'OECDE', 'AUS', 'OECD',
       'SWE', 'MEX', 'GBR', 'ZAF', 'USA', 'HUN', 'PRT', 'DNK', 'ESP',
       'LUX', 'GRC', 'BRA', 'SVK', 'CHN', 'BEL', 'FIN', 'NZL', 'G-7',
       'IDN', 'TUR', 'AUT', 'ITA', 'IRL', 'SVN', 'DEU', 'KOR', 'EST',
       'EA19', 'ISR', 'RUS', 'LVA', 'LTU', 'G7M', 'OEU', 'COL', 'CHL',
       'CRI', 'IND', 'G-20', 'EU27_2020'], dtype=object)

#### Selecting only domestic data

In [8]:
# Selecting uneeded features from `df_cci`
drop_cols = [
    'LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Flag Codes'
]

# Copying domestic data from `df_cci` to `df_cci_us` and removing uneeded features
df_cci_us = df_cci.loc[df_cci['LOCATION'] == 'USA'].drop(columns=drop_cols).reset_index(drop=True).copy()

# Confirming total records for `df_cci_us`
display(df_cci_us.shape)
display(df_cci_us.head(3))
display(df_cci_us.tail(3))

(766, 2)

,TIME,Value
0,1960-01,101.6304
1,1960-02,101.3717
2,1960-03,101.1483


,TIME,Value
763,2023-08,97.98888
764,2023-09,97.82825
765,2023-10,97.52642


#### Data conversion

In [9]:
# Converting `TIME` to datetime
df_cci_us['TIME'] = pd.to_datetime(df_cci_us['TIME'])

# Renaming `TIME` to `Date` and `Value` to `CCI Value` for readability after concatenation
df_cci_us.rename(columns={'TIME': 'Date', 'Value': 'CCI Value'}, inplace=True)

#### Indexing

In [10]:
# Setting `Date` as the index for `df_cci_us`
df_cci_us.set_index('Date', inplace=True)

# Confirming `df_cci_us` ready to concatenate
df_cci_us.head()

,CCI Value
Date,
1960-01-01,101.6304
1960-02-01,101.3717
1960-03-01,101.1483
1960-04-01,101.0250
1960-05-01,101.0561


# Inflation

#### Preprocessing of the `US_inflation_rates.csv` dataset

Seeing as the dataset came with only the needed features, little will be needed to prepare the DF for concatenation with the other economic datasets. `date` will be converted to datetime and set as the index.

In [11]:
# Viewing `df_inflation`
df_inflation.head()

,date,value
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


#### Beginning on limited EDA

In [12]:
# Beginning of EDA on `df_inflation`
df_inflation.describe()

,value
count,918.000000
mean,117.794828
std,83.736360
min,21.480000
25%,32.105000
50%,106.900000
75%,188.000000
max,303.841000


In [13]:
# Continuing EDA
df_inflation.shape

(918, 2)

In [14]:
# Continuing EDA
df_inflation.dtypes

date      object
value    float64
dtype: object

#### Data conversion

In [15]:
# Converting `date` to datetime
df_inflation['date'] = pd.to_datetime(df_inflation['date'])

# Renaming `date` to `Date` and `value` to `CPI Value` for readability after concatenation
df_inflation.rename(columns={'date': 'Date', 'value': 'CPI Value'}, inplace=True)

#### Indexing

In [16]:
# Setting `Date` as the index for `df_inflation`
df_inflation.set_index('Date', inplace=True)

# Confirming `df_inflation` ready to concatenate
df_inflation.head()

,CPI Value
Date,
1947-01-01,21.48
1947-02-01,21.62
1947-03-01,22.00
1947-04-01,22.00
1947-05-01,21.95


# Public Debt

#### Preprocessing of the `public_debt.csv` dataset

Since this datset came as quarterly values, both converting `date` to datetime and the index will be necessary to resample for the missing months. Once the values are forward filled, the DF will be ready to concatenate with the other economic datasets.

In [17]:
# Viewing `df_public_debt`
df_public_debt.head()

,date,value
0,1966-01-01,320999.0
1,1966-04-01,316097.0
2,1966-07-01,324748.0
3,1966-10-01,329319.0
4,1967-01-01,330947.0


#### Beginning of limited EDA

In [18]:
# Beginning of EDA on `df_public_debt`
df_public_debt.describe()

,value
count,2.290000e+02
mean,7.408932e+06
std,8.189013e+06
min,3.160970e+05
25%,8.776140e+05
50%,4.692750e+06
75%,1.069980e+07
max,3.145844e+07


In [19]:
# Continuing EDA
df_public_debt.shape

(229, 2)

In [20]:
# Continuing EDA
df_public_debt.dtypes

date      object
value    float64
dtype: object

#### Data conversion

In [21]:
# Converting `date` to datetime
df_public_debt['date'] = pd.to_datetime(df_public_debt['date'])

# Renmaing `date` to `Date` and `value` to `Quarterly Public Debt (bil USD)` for readability after concatenation
df_public_debt.rename(columns={'date': 'Date', 'value': 'Quarterly Public Debt (bil USD)'}, inplace=True)

#### Indexing

In [22]:
# Setting `Date` as the index for `df_public_debt`
df_public_debt.set_index('Date', inplace=True)

#### Resampling the quarters

In [23]:
# Filling data for each quarter of `Date`
df_public_debt_quarters = df_public_debt.resample('MS').ffill()

# Confirming `df_public_debt_quarters` ready to concantate
df_public_debt_quarters.head(10)

,Quarterly Public Debt (bil USD)
Date,
1966-01-01,320999.0
1966-02-01,320999.0
1966-03-01,320999.0
1966-04-01,316097.0
1966-05-01,316097.0
1966-06-01,316097.0
1966-07-01,324748.0
1966-08-01,324748.0
1966-09-01,324748.0


## Hourly Wages

#### Preprocessing of the `Medianaverage hourly wages.csv` dataset

This dataset came with uneeded features that will need to be dropped, as well as the needed features will need to be converted to `float`. Additionally, as an annual measures of mean and median wages, the `Date` feature will need to be converted to datetime and set to the index in order to resample for the missing months. Once the values are forward filled, the DF will be ready to concatenate with with other economic datasets.

In [24]:
# Viewing `df_hourly_wages`
df_hourly_wages.head()

,Date,Median,Average,Men Median,Men Average,Women Median,Women Average,White Median,White Average,Black Median,...,Black Women Median,Black Women Average,Hispanic Women Median,Hispanic Women Average,Recent high-school graduate,Men Recent high-school graduate,Women Recent high-school graduate,Recent college graduate,Men Recent college graduate,Women Recent college graduate
0,2023,$23.75,$33.50,$25.06,$36.98,$21.80,$29.74,$25.97,$36.09,$20.11,...,$19.93,$25.59,$18.30,$23.60,$16.16,$17.11,$14.76,$25.93,$28.59,$24.01
1,2022,$23.31,$33.31,$25.29,$37.29,$21.08,$29.00,$25.96,$35.90,$20.39,...,$19.75,$24.98,$18.28,$23.53,$16.48,$17.47,$15.00,$25.14,$27.48,$23.41
2,2021,$23.74,$33.43,$25.92,$37.11,$21.85,$29.46,$26.49,$35.94,$20.26,...,$19.69,$25.14,$18.48,$23.86,$15.87,$16.46,$15.02,$26.10,$28.97,$24.03
3,2020,$24.09,$33.95,$26.56,$37.60,$22.53,$30.01,$27.22,$36.38,$20.79,...,$20.05,$25.95,$18.29,$24.03,$15.29,$16.11,$14.11,$27.38,$28.68,$26.46
4,2019,$22.94,$31.71,$24.75,$35.14,$21.01,$28.02,$25.49,$34.24,$19.19,...,$18.88,$24.20,$17.92,$22.45,$15.34,$16.21,$14.08,$24.97,$27.08,$23.46


#### Beginning of limited EDA

In [25]:
# Beginning EDA on `df_hourly_wages`
df_hourly_wages.describe()

,Date
count,51.000000
mean,1998.000000
std,14.866069
min,1973.000000
25%,1985.500000
50%,1998.000000
75%,2010.500000
max,2023.000000


In [26]:
# Continuing EDA
df_hourly_wages.shape

(51, 31)

In [27]:
# Continuing EDA
df_hourly_wages.dtypes

Date                                  int64
Median                               object
Average                              object
Men Median                           object
Men Average                          object
Women Median                         object
Women Average                        object
White Median                         object
White Average                        object
Black Median                         object
Black Average                        object
Hispanic Median                      object
Hispanic Average                     object
White Men Median                     object
White Men Average                    object
Black Men Median                     object
Black Men Average                    object
Hispanic Men Median                  object
Hispanic Men Average                 object
White Women Median                   object
White Women Average                  object
Black Women Median                   object
Black Women Average             

#### Selecting only needed features

In [28]:
# Selecting needed features from `df_hourly_wafges`
retain_cols = [
    'Date', 'Median', 'Average'
]

# Copying data from `df_hourly_wages` to `df_hourly_wages_avgs` and removing uneeded features
df_hourly_wages_avgs = df_hourly_wages.drop(columns=[
    col for col in df_hourly_wages.columns if col not in retain_cols
]).copy()

#### Data Conversion

In [29]:
# The `Median` and `Average` features will need the '$' stripped from the data
# and the features will need to be converted to `float`

# Selecting features to be stripped and converted
dollar_cols = [
    'Median', 'Average'
]

# Defining a function to remove '$' and convert data to `float`
def convert_dollars(val):
    if isinstance(val, str):
        return float(val.strip('$'))
    return val

# Applying function to `dollar_cols` in `df_hourly_wages_avgs`
for col in df_hourly_wages_avgs[dollar_cols].columns:
    try:
        df_hourly_wages_avgs[col] = df_hourly_wages_avgs[col].apply(convert_dollars)
    except ValueError:
        print(f'Column {col} could not be converted, please verify dtype')

# Renaming `Meadian` to `Annual Hourly Median Wage (USD)`
# and `Average` to `Annual Hourly Mean Wage (USD)` for readability after concatenation
df_hourly_wages_avgs.rename(columns={
    'Median': 'Annual Hourly Median Wage (USD)',
    'Average': 'Annual Hourly Mean Wage (USD)'
}, inplace=True)

In [30]:
# Converting `Date` to datetime
df_hourly_wages_avgs['Date'] = pd.to_datetime(df_hourly_wages_avgs['Date'], format='%Y')

#### Indexing

In [31]:
# Setting `Date` as the index for `df_hourly_wages_avgs`
df_hourly_wages_avgs.set_index('Date', inplace=True)

#### Resampling the years

In [32]:
# Filling data for each month of `Date`
df_hourly_wages_months = df_hourly_wages_avgs.resample('MS').ffill()

# Confirming `df_hourly_wages_mos` ready to concatenate
df_hourly_wages_months.head(13)

,Annual Hourly Median Wage (USD),Annual Hourly Mean Wage (USD)
Date,,
1973-01-01,20.58,24.03
1973-02-01,20.58,24.03
1973-03-01,20.58,24.03
1973-04-01,20.58,24.03
1973-05-01,20.58,24.03
1973-06-01,20.58,24.03
1973-07-01,20.58,24.03
1973-08-01,20.58,24.03
1973-09-01,20.58,24.03


## Unemployment

#### Preprocessing of the `Unemployment.csv` dataset

This dataset came with uneeded features that will need to be dropped, as well as the needed features will need to be converted to `float`. Additionally, the `Date` feature will need to be converted to datetime and set to the index in preparation for concatenation with the other economic datasets.

In [33]:
# Viewing `df_unemp`
df_unemp.head()

,Date,All,Women,Men,Black,Hispanic,White,Black Women,Black Men,Hispanic Women,...,White Men Some college,16-24,25-54,55-64,65+,Advanced degree,Bachelor's degree,High school,Less than HS,Some college
0,Mar-2024,3.7%,3.5%,3.9%,5.8%,4.6%,3.0%,5.5%,6.2%,4.6%,...,2.9%,8.0%,3.2%,2.6%,3.0%,2.0%,2.5%,5.0%,7.7%,3.4%
1,Feb-2024,3.7%,3.5%,3.8%,5.7%,4.6%,3.0%,5.4%,6.0%,4.6%,...,2.9%,7.9%,3.2%,2.6%,2.9%,1.9%,2.5%,5.0%,7.5%,3.4%
2,Jan-2024,3.7%,3.5%,3.8%,5.7%,4.7%,2.9%,5.4%,5.9%,4.6%,...,2.9%,7.9%,3.1%,2.6%,2.9%,1.9%,2.4%,4.9%,7.4%,3.4%
3,Dec-2023,3.6%,3.5%,3.8%,5.7%,4.6%,2.9%,5.5%,5.9%,4.6%,...,2.9%,7.9%,3.1%,2.5%,2.8%,1.9%,2.5%,4.9%,7.4%,3.4%
4,Nov-2023,3.6%,3.4%,3.8%,5.7%,4.6%,2.9%,5.6%,5.9%,4.5%,...,2.9%,8.0%,3.1%,2.5%,2.8%,1.9%,2.5%,4.8%,7.3%,3.4%


#### Beginning of limited EDA

In [34]:
# Beginning EDA on `df_unemp`
df_unemp.describe()

,Date,All,Women,Men,Black,Hispanic,White,Black Women,Black Men,Hispanic Women,...,White Men Some college,16-24,25-54,55-64,65+,Advanced degree,Bachelor's degree,High school,Less than HS,Some college
count,544,544,544,544,544,544,544,544,544,544,...,544,544,544,544,544,544,544,544,544,544
unique,544,68,65,71,132,99,60,130,142,98,...,62,108,58,50,51,27,43,80,112,64
top,Mar-2024,5.3%,5.4%,5.2%,10.5%,10.5%,4.2%,9.8%,11.4%,11.0%,...,3.7%,10.9%,4.2%,3.1%,3.1%,2.0%,2.5%,5.4%,9.9%,4.7%
freq,1,26,33,22,18,17,23,14,15,29,...,39,19,34,41,53,72,55,28,20,24


In [35]:
# Continuing EDA
df_unemp.shape

(544, 121)

In [36]:
# Continuing EDA
df_unemp.dtypes

Date                 object
All                  object
Women                object
Men                  object
Black                object
                      ...  
Advanced degree      object
Bachelor's degree    object
High school          object
Less than HS         object
Some college         object
Length: 121, dtype: object

#### Selecting only needed features

In [37]:
# Selecting needed features from `df_unemp`
retain_cols = ['Date', 'All']

# Copying data from `df_unemp` to `df_unemp_one` and removing uneeded features
df_unemp_one = df_unemp.drop(columns=[
    col for col in df_unemp.columns if col not in retain_cols
]).copy()

#### Data conversion

In [38]:
# The `All` feature will need the '%' stripped from the data and the feature
# will need to be converted to a `float`

# Defining a function to remove '%' and convert data to numeric percentages
def convert_percentage(obj):
    return float(obj.strip('%'))

# Applying function to `agg_cols` in `df_unemp`
for col in df_unemp_one.columns:
    try:
        df_unemp_one[col] = df_unemp_one[col].apply(convert_percentage)
    except ValueError:
        print(f'Column {col} could not be converted, please verify dtype')

# Renaming `All` to `Monthly Unemployment Rate (%)` for readability after concatenation
df_unemp_one.rename(columns={
    'All': 'Monthly Unemployment Rate (%)'
}, inplace=True)

Column Date could not be converted, please verify dtype


In [39]:
df_unemp_one.head()

,Date,Monthly Unemployment Rate (%)
0,Mar-2024,3.7
1,Feb-2024,3.7
2,Jan-2024,3.7
3,Dec-2023,3.6
4,Nov-2023,3.6


In [40]:
# The `Date` feature will need to be engineered into a workable datetime feature

# Creating a dictionary of Months
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Mapping integer month values to `Date Month`
df_unemp_one['Date Month'] = df_unemp_one['Date'].str.slice(0,3).map(month_map)

# Slicing `Date Year`
df_unemp_one['Date Year'] = df_unemp_one['Date'].str.slice(4,8)

# Converting `Date` to datetime using `Date Month` and `Date Year`
df_unemp_one['Date'] = pd.to_datetime({
    'year': df_unemp_one['Date Year'],
    'month': df_unemp_one['Date Month'],
    'day': 1
})

# Dropping engineered features `Date Month` and `Date Year`
df_unemp_one.drop(columns=['Date Month', 'Date Year'], inplace=True)

#### Indexing

In [41]:
# Setting `Date` as the index for `df_unemp_one`
df_unemp_one.set_index('Date', inplace=True)

# Confirming `df_unemp_one` ready to concatenate
df_unemp_one.head(13)

,Monthly Unemployment Rate (%)
Date,
2024-03-01,3.7
2024-02-01,3.7
2024-01-01,3.7
2023-12-01,3.6
2023-11-01,3.6
2023-10-01,3.6
2023-09-01,3.6
2023-08-01,3.6
2023-07-01,3.6


## Combined Economics

#### Preprocessing of the `df_economics` DF

With all datasets set to a monthly datetime index, the relevent features of all can be combined into one DF, and any NaN records can be dropped.

In [45]:
# Concatenating the five economic datasets into `df_economics`
df_economics = pd.concat(
    [
        df_cci_us,
        df_inflation,
        df_public_debt_quarters,
        df_hourly_wages_months,
        df_unemp_one
    ], axis=1, join='outer'
)

#### Handling for `NaN` rows

In [47]:
# Confirming total records
df_economics.shape

(927, 6)

In [48]:
# Checking total `NaN` rows
df_economics.isna().sum()

CCI Value                          161
CPI Value                            9
Quarterly Public Debt (bil USD)    242
Annual Hourly Median Wage (USD)    326
Annual Hourly Mean Wage (USD)      326
Monthly Unemployment Rate (%)      383
dtype: int64

In [51]:
# Dropping `NaN` rows
df_economics.dropna(inplace=True)

In [52]:
# Confirming remaining records
df_economics.shape

(530, 6)

In [55]:
# Confirming final economic DF
display(df_economics.head())
display(df_economics.tail())

,CCI Value,CPI Value,Quarterly Public Debt (bil USD),Annual Hourly Median Wage (USD),Annual Hourly Mean Wage (USD),Monthly Unemployment Rate (%)
Date,,,,,,
1978-12-01,98.26257,67.9,789207.0,20.41,23.77,6.1
1979-01-01,98.23715,68.5,796792.0,19.86,23.98,6.1
1979-02-01,98.22179,69.2,796792.0,19.86,23.98,6.0
1979-03-01,98.03010,69.9,796792.0,19.86,23.98,6.0
1979-04-01,97.83338,70.6,804913.0,19.86,23.98,5.9


,CCI Value,CPI Value,Quarterly Public Debt (bil USD),Annual Hourly Median Wage (USD),Annual Hourly Mean Wage (USD),Monthly Unemployment Rate (%)
Date,,,,,,
2022-09-01,96.65869,296.539,30928912.0,23.31,33.31,3.8
2022-10-01,96.77125,297.987,31419689.0,23.31,33.31,3.7
2022-11-01,96.83717,298.598,31419689.0,23.31,33.31,3.7
2022-12-01,97.05321,298.990,31419689.0,23.31,33.31,3.6
2023-01-01,97.33643,300.536,31458438.0,23.75,33.50,3.6


## Proof of Concept

#### Engineering the economic target value

As a feature will need to be engineered for the final modelling, the following cells will be used to test the scaling and thresholds for classification as a proof of concept prior to merging the economic and movie datasets. A working DF will be used so that `df_economics` may be retained and untouched for modeling as a complete dataset.

For the purposes of this feature engineering, we will be looking to classify `Economic State` by the following definitions;

* Boom - Strong economic growth, low unemployment, high consumer confidence, and stable or increasing wages
* Expansion - Economic growth is positive, but not as robust as during a Boom
* Recession - Economic growth is negative, rising unemployment, and decreasing consumer confidence
* Contraction - Economic growth is negative, but not as severe as a Recession
* Recovery - Economic growth is starting to pick up after a Recession or Contraction
* Stagnation - Economic grown is very low or unmoving, with high unemployment and low consumer confidence

Given the data present in the `df_economics` dataset, we can interpret the values and trends in rolling windows to make this classification possible.

* **CCI Value**: Values over `100` indicate stronger consumer confidence, and values below `100` indicate stronger consumer caution. Trends will help distinguish between `Boom`, `Recession`, `Stagnation`, and the states between.
* **CPI Value**: Represents inflation. Increasing CPI might indicate inflationary pressures seen during Boom periods, while stable or decreasing CPI could indicate lower economic activity associated with Recession or Contraction.
* **Quarterly Public Debt**: Higher public debt might indicate economic stress and government borrowing to stimulate the economy, often seen in Recession or Contraction. Lower or stable public debt might be seen in Boom or Expansion periods.
* **Annual Hourly Median Wage**: Increasing median wages are associated with Boom or Expansion, while stagnant or decreasing median wages might be seen in Recession or Contraction.
* **Annual Hourly Mean Wage**: Similar to median wages, increasing mean wages indicate Boom or Expansion, while stagnant or decreasing mean wages might be seen in Recession or Contraction.
* **Monthly Unemployment Rate**: Lower unemployment rates are associated with Boom or Expansion, while higher unemployment rates are seen in Recession or Contraction.

By using a rolling 12-month window, we can illustrate the trends for each feature in any given month, and create the feature `Economic State`.

#### Continued EDA

In [69]:
# Creating the working DF `df_eco_test` as a copy of `df_economics`
df_eco_test = df_economics.copy()

# Viewing `df_eco_test`
df_eco_test.head()

,CCI Value,CPI Value,Quarterly Public Debt (bil USD),Annual Hourly Median Wage (USD),Annual Hourly Mean Wage (USD),Monthly Unemployment Rate (%)
Date,,,,,,
1978-12-01,98.26257,67.9,789207.0,20.41,23.77,6.1
1979-01-01,98.23715,68.5,796792.0,19.86,23.98,6.1
1979-02-01,98.22179,69.2,796792.0,19.86,23.98,6.0
1979-03-01,98.03010,69.9,796792.0,19.86,23.98,6.0
1979-04-01,97.83338,70.6,804913.0,19.86,23.98,5.9


In [70]:
# Continuing EDA
df_eco_test.describe()

,CCI Value,CPI Value,Quarterly Public Debt (bil USD),Annual Hourly Median Wage (USD),Annual Hourly Mean Wage (USD),Monthly Unemployment Rate (%)
count,530.000000,530.000000,5.300000e+02,530.000000,530.000000,530.000000
mean,99.947584,176.117704,9.347157e+06,21.167547,27.086547,6.189245
std,1.526357,57.728120,8.163929e+06,1.200100,2.933874,1.619744
min,96.182430,67.900000,7.892070e+05,19.590000,23.370000,3.600000
25%,98.737980,126.600000,2.977735e+06,20.080000,24.352500,4.925000
50%,100.479450,175.100000,5.773566e+06,21.270000,27.230000,5.800000
75%,101.013025,227.687250,1.551062e+07,22.087500,28.720000,7.300000
max,102.850100,300.536000,3.145844e+07,24.090000,33.950000,10.300000


In [71]:
# Continuing EDA
df_eco_test.shape

(530, 6)

#### Classifying `Economic State`

In [74]:
# Creating a list of features in `df_eco_test`
agg_cols = df_eco_test.columns.to_list()

# Mapping columns to their corresponding rolling mean names
rolling_mean_mapping = {
    'CCI Value': 'CCI Rolling Mean',
    'CPI Value': 'CPI Rolling Mean',
    'Quarterly Public Debt (bil USD)': 'Pub Debt Rolling Mean',
    'Annual Hourly Median Wage (USD)': 'Med Wage Rolling Mean',
    'Annual Hourly Mean Wage (USD)': 'Mean Wage Rolling Mean',
    'Monthly Unemployment Rate (%)': 'Unemp Rolling Mean'
}

# Calculating rolling means for each feature of `df_eco_test`
for col in agg_cols:
    if col in rolling_mean_mapping:
        rolling_mean_col = rolling_mean_mapping[col]
        df_eco_test[rolling_mean_col] = df_eco_test[col].rolling(window=12).mean()

In [84]:
# Creating a list of features in `df_eco_test`
pct_cols = df_eco_test[[
    'CCI Rolling Mean',
    'CPI Rolling Mean',
    'Pub Debt Rolling Mean',
    'Med Wage Rolling Mean',
    'Mean Wage Rolling Mean',
    'Unemp Rolling Mean'
]].columns.to_list()

# Mapping columns to their corresponding rolling mean names
rolling_pct_mapping = {
    'CCI Rolling Mean': 'CCI Rolling Percet Change',
    'CPI Rolling Mean': 'CPI Rolling Percet Change',
    'Pub Debt Rolling Mean': 'Pub Debt Rolling Percet Change',
    'Med Wage Rolling Mean': 'Med Wage Rolling Percet Change',
    'Mean Wage Rolling Mean': 'Mean Wage Rolling Percet Change',
    'Unemp Rolling Mean': 'Unemp Rolling Percet Change'
}

# Calculating rolling means for each feature of `df_eco_test`
for col in pct_cols:
    if col in rolling_pct_mapping:
        rolling_pct_col = rolling_pct_mapping[col]
        df_eco_test[rolling_pct_col] = df_eco_test[col].pct_change(periods=12) * 100

In [85]:
df_eco_test[[
    'CCI Rolling Percet Change',
    'CPI Rolling Percet Change',
    'Pub Debt Rolling Percet Change',
    'Med Wage Rolling Percet Change',
    'Mean Wage Rolling Percet Change',
    'Unemp Rolling Percet Change'
]].describe()

,CCI Rolling Percet Change,CPI Rolling Percet Change,Pub Debt Rolling Percet Change,Med Wage Rolling Percet Change,Mean Wage Rolling Percet Change,Unemp Rolling Percet Change
count,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000
mean,0.009669,3.188011,8.917386,0.404551,0.812323,1.057584
std,0.919494,2.179986,4.646751,1.424076,1.396320,18.727424
min,-2.603210,-0.621247,-0.239135,-2.615012,-2.569249,-39.069767
25%,-0.396458,1.909764,5.358954,-0.635353,0.043179,-9.264461
50%,0.041793,2.791425,8.542081,0.256644,0.633627,-5.798687
75%,0.547931,3.638977,11.948413,1.316574,1.672321,9.330822
max,2.750711,13.584687,19.892008,5.013078,7.064018,95.805740


In [60]:
# Defining the thresholds for trends
trend = {
    'str_pos': #0.05, # 5% or more increase, strong positive trend
    'mod_pos': #0.02, # 2% to 5% increase, moderate positive trend
    'neutral': #0.00, # No significant change, neutral trend
    'mod_neg': #-0.02, # -2% to -5% decrease, moderate negative trend
    'str_neg': #-0.05 # -5% or more decrease, strong negative trend
}